# Processing MCSL summer swimming result

In [1]:
from bs4 import BeautifulSoup
import csv
import os
import glob
import pandas as pd

In [2]:
#file/folder
htmlfile = "./data/2019/week1"
filename = 'WvEW.html'
path = os.path.join(htmlfile, f'{filename}')
print(path)

./data/2019/week1/WvEW.html


In [3]:
with open(path, 'r') as f:
    contents = f.read()
    soup = BeautifulSoup(contents, 'lxml')

#print(soup.prettify())

In [4]:
htmlfile_list = htmlfile.split('/')
year = htmlfile_list[2]
week = htmlfile_list[3]

In [5]:
table = soup.find("table")
cols = table.findAll("td")

content = []
result = []
record = []

for td in cols:
    try:
        text = ''.join(td.find(text=True))
        content.append(text)
    except: pass

event = content[0]

#loop the rest
for line in content:
    if line[0:5] == 'Event':
        event = line
    else:
        #start of new record, when it sees rank
        if (line[1:2] == '.' and len(line)==2) or (line[2:3] == '.' and len(line) == 3):
            if result:
                record.append(result)
                result = []
            result.append(year)
            result.append(week)
            result.append(event)
            result.append(line)
        else:
            result.append(line)



In [6]:
#take out relay record, all individual records has 7 columns
individual_record = []
for l in record:
    if len(l)==7:
        #print(l)
        individual_record.append(l)
        

In [7]:
df = pd.DataFrame(individual_record, columns =['year', 'week', 'event', 'rank', 'swimmer', 'seed', 'final']) 

In [8]:
df.head(2)

,year,week,event,rank,swimmer,seed,final
0,2019,week1,Event 1 - Boys 12U 100M Individual Medley,1.,"Manning, Jackson R (12) (W)",1:39.31,1:30.47
1,2019,week1,Event 1 - Boys 12U 100M Individual Medley,2.,"Chu-Roxas, Aidan (12) (EW)",1:35.84,1:34.97


In [9]:
#splitting swimmer colunn into 3
df[['swimmer_name', 'swimmer_age', 'swimmer_team']] = df['swimmer'].str.split(pat='(', expand=True)

#dropping progoma; swimmer column
del df['swimmer']

In [10]:
#remove ')' from age and team
df['swimmer_age'] = df['swimmer_age'].apply(lambda x:x.replace(')',''))
df['swimmer_team'] = df['swimmer_team'].apply(lambda x:x.replace(')',''))

In [11]:
df.head(2).T

,0,1
year,2019,2019
week,week1,week1
event,Event 1 - Boys 12U 100M Individual Medley,Event 1 - Boys 12U 100M Individual Medley
rank,1.,2.
seed,1:39.31,1:35.84
final,1:30.47,1:34.97
swimmer_name,"Manning, Jackson R","Chu-Roxas, Aidan"
swimmer_age,12,12
swimmer_team,W,EW
